In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load in csv
df = pd.read_csv(r"../../data/jena_climate_2009_2016.csv",
                parse_dates=['Date Time'],
                index_col=['Date Time'])
df.index = pd.to_datetime(df.index, format='%d.%m.%Y %H:%M:%S')
# For these tests we will just use a small slice of the dataset
df = df.iloc[:1000,:]

In [240]:
class CoilNormalizer():
    def __init__(self):
        super(CoilNormalizer, self).__init__()
        self.max_change_df = None
        self.conserved_subgroups = None
    
    def max_absolute_change(self, df_diff):
        # Calculate the absolute change for each feature
        abs_change = df_diff.abs()

        # Find the maximum absolute change for each feature
        max_changes = abs_change.max()

        # Convert the series to a DataFrame
        max_change_df = max_changes.to_frame(name='Max Absolute Change')

        return max_change_df
    
    def normalize(self, df):
        # Calculate the change for each feature
        df_diff = df.diff()

        # Drop first row
        df_diff = df_diff.iloc[1:,:]
        
        # Calculate the max absolute change for each feature
        max_change_df = self.max_absolute_change(df_diff)

        normalized_features = []
        conserved_subgroups = {}
        count = 0
        index = 0
        for column in df_diff.columns:
            max_change_val = max_change_df.loc[column, 'Max Absolute Change']
            normalized_feature = (df_diff[column] + max_change_val) / (2 * max_change_val)
            
            normalized_counter = 1 - normalized_feature
            
            normalized_features.append(normalized_counter)
            normalized_features.append(normalized_feature)
            
            conserved_subgroups[index] = [count, count + 1]
            index += 1
            count += 2
            
        # Create dataframe
        normalized_df = pd.DataFrame(normalized_features).T
        print(normalized_df)
                
        # Normalize the dataframe again by row such that each row sums to 1
        normalized_df = normalized_df.div(normalized_df.sum(axis=1), axis=0)
        
        self.max_change_df = max_change_df
        self.conserved_subgroups = conserved_subgroups
        return normalized_df
    
    def denormalize(self, normalized_df, initial_value):
        # For now assume there are even pairs
        denorm_dict = {}
        for key, value in self.conserved_subgroups.items():
            df_sel = normalized_df.iloc[:,value]
            denorm_dict[df_sel.columns[1]] = df_sel.div(df_sel.sum(axis=1), axis=0).iloc[:,1]
            
        denormalized_df = pd.DataFrame(denorm_dict, index=normalized_df.index)
        
        reconstructed_array = []
        new_value = initial_value
        reconstructed_array.append(initial_value)
        delta_max = self.max_change_df.iloc[:,0]
        for i in range(denormalized_df.shape[0]):
            delta_t = 2 * denormalized_df.iloc[i,:] * delta_max - delta_max
            new_value = new_value + delta_t
            reconstructed_array.append(new_value)
        return pd.DataFrame(reconstructed_array, index = [initial_value.name] + list(denormalized_df.index))



In [241]:
# Instantiate CoilNormalizer
coilnormer = CoilNormalizer()

coilnormed_df = coilnormer.normalize(df)

                     p (mbar)  p (mbar)  T (degC)  T (degC)  Tpot (K)  \
Date Time                                                               
2009-01-01 00:20:00  0.413793  0.586207  0.634483  0.365517  0.633562   
2009-01-01 00:30:00  0.568966  0.431034  0.534483  0.465517  0.534247   
2009-01-01 00:40:00  0.534483  0.465517  0.431034  0.568966  0.428082   
2009-01-01 00:50:00  0.500000  0.500000  0.486207  0.513793  0.489726   
2009-01-01 01:00:00  0.517241  0.482759  0.424138  0.575862  0.421233   
...                       ...       ...       ...       ...       ...   
2009-01-07 22:00:00  0.310345  0.689655  0.272414  0.727586  0.277397   
2009-01-07 22:10:00  0.206897  0.793103  0.462069  0.537931  0.465753   
2009-01-07 22:20:00  0.155172  0.844828  0.703448  0.296552  0.708904   
2009-01-07 22:30:00  0.327586  0.672414  0.689655  0.310345  0.691781   
2009-01-07 22:40:00  0.293103  0.706897  0.634483  0.365517  0.636986   

                     Tpot (K)  Tdew (degC)  Tdew (

In [242]:
coilnormed_df.sum(axis =1)

Date Time
2009-01-01 00:20:00    1.0
2009-01-01 00:30:00    1.0
2009-01-01 00:40:00    1.0
2009-01-01 00:50:00    1.0
2009-01-01 01:00:00    1.0
                      ... 
2009-01-07 22:00:00    1.0
2009-01-07 22:10:00    1.0
2009-01-07 22:20:00    1.0
2009-01-07 22:30:00    1.0
2009-01-07 22:40:00    1.0
Length: 999, dtype: float64

In [243]:
initial_value = df.iloc[0,:]
coilnormer.denormalize(coilnormed_df,initial_value)

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
2009-01-01 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
2009-01-01 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2009-01-01 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
2009-01-01 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
2009-01-01 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-01-07 22:00:00,997.11,-9.41,263.97,-10.27,93.4,2.99,2.79,0.20,1.74,2.80,1315.59,0.52,1.38,188.1
2009-01-07 22:10:00,997.28,-9.30,264.07,-10.28,92.5,3.01,2.79,0.23,1.74,2.80,1315.27,0.29,1.13,85.4
2009-01-07 22:20:00,997.48,-9.89,263.46,-10.96,91.8,2.88,2.64,0.24,1.65,2.65,1318.56,0.29,1.00,119.3
2009-01-07 22:30:00,997.58,-10.44,262.90,-11.59,91.2,2.75,2.51,0.24,1.57,2.52,1321.51,0.55,1.00,130.5


In [244]:
df

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
Date Time,,,,,,,,,,,,,,
2009-01-01 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
2009-01-01 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2009-01-01 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
2009-01-01 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
2009-01-01 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-01-07 22:00:00,997.11,-9.41,263.97,-10.27,93.4,2.99,2.79,0.20,1.74,2.80,1315.59,0.52,1.38,188.1
2009-01-07 22:10:00,997.28,-9.30,264.07,-10.28,92.5,3.01,2.79,0.23,1.74,2.80,1315.27,0.29,1.13,85.4
2009-01-07 22:20:00,997.48,-9.89,263.46,-10.96,91.8,2.88,2.64,0.24,1.65,2.65,1318.56,0.29,1.00,119.3


In [247]:
# We should be able to take any slice of the coilnormed timeseries and reproduce
start_index = 200
end_index = 300
df_orig_slice = df.iloc[start_index:end_index,:]
coilnormed_df_slice = coilnormed_df.iloc[start_index:end_index -1,:]
initial_value_slice = df.iloc[start_index,:]

denormed_slice = coilnormer.denormalize(coilnormed_df_slice,initial_value_slice)

In [248]:
import plotly.graph_objects as go
import pandas as pd

# Sample data creation
df1 = denormed_slice
df2 = df_orig_slice

# Plotting
fig = go.Figure()

# Add traces for the first dataframe
for column in df1.columns:
    fig.add_trace(go.Scatter(x=df2.index, y=df1[column], mode='lines', name=f'Denormed: {column}'))

# Add traces for the second dataframe
for column in df2.columns:
    fig.add_trace(go.Scatter(x=df2.index, y=df2[column], mode='lines', name=f'Orig: {column}'))

# Update layout
fig.update_layout(title='Interactive Time Series Plot',
                  xaxis_title='Date',
                  yaxis_title='Value',
                  legend_title='Legend',
                  hovermode='x unified')

# Show the plot
fig.show()
